# Model comparison

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklego.metrics import equal_opportunity_score
from sklego.metrics import p_percent_score
from sklearn.metrics import log_loss
from sklearn.utils.extmath import squared_norm
from moopt.scalarization_interface import scalar_interface, single_interface, w_interface
from moopt import monise
import numpy as np
import optuna, sklearn, sklearn.datasets
from fair_models import coefficient_of_variation, MOOLogisticRegression, FindCLogisticRegression, FindCCLogisticRegression

Using Python-MIP package version 1.7.2


/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Parte 1: Data treatment

In [3]:
mydata= pd.read_csv("Datasets/german_credit_data.csv")

Dados de pedidos de crédito. É um dos datasets mais utilizados para tutoriais em Fairness, como na biblioteca [$aif360$](https://github.com/IBM/AIF360/blob/master/examples/README.md). Dataset original disponível em [aqui](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)), mas eu utilizei [este](https://www.kaggle.com/kabure/german-credit-data-with-risk), por estar em formato csv com os headers, embora omita informações do dataset original.

Originalmente possui 1000 dados, mas com vários valores NaN, ficando com 522 dados após remoção de dados com informações faltantes.

In [4]:
mydata.head()

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [5]:
mydata = mydata.drop(['Unnamed: 0', 'Purpose'], axis=1)

In [6]:
mydata = mydata.dropna()

In [7]:
mapping_Sex = {'male': 0, 'female': 1}
mapping_Housing = {'free': 1, 'rent': 2, 'own': 3}
mapping_Savings = {'little': 1, 'moderate': 2, 'quite rich': 3, 'rich': 4}
mapping_Checking = {'little': 1, 'moderate': 2, 'rich': 3}
mapping_Risk = {"bad": -1, "good": 1}

numerical_data = mydata.replace({'Sex': mapping_Sex, 'Housing': mapping_Housing, 'Saving accounts': mapping_Savings,
                'Checking account':mapping_Checking, 'Risk': mapping_Risk})

In [8]:
X = numerical_data.drop(['Risk'], axis=1)

In [9]:
X.shape

(522, 8)

In [10]:
y = numerical_data['Risk']

In [11]:
# 395, 808
random_seed = 2000#np.random.randint(0, 1000)
random_seed

2000

In [12]:
random_seed2 = 2000#np.random.randint(0, 1000)
random_seed2

2000

In [13]:
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=200, random_state = random_seed)
X_train, X_val, y_train, y_val = train_test_split(X_tv, y_tv, test_size=100, random_state = random_seed2)

In [14]:
def calc_reweight(X, y):
    W = {}
    W[0] = {}
    W[1] = {}

    D = len(X)
    len_men = X.groupby('Sex').count()['Age'][0]
    len_women = X.groupby('Sex').count()['Age'][1]
    len_neg = sum(y==-1)
    len_pos = sum(y==1)
    len_men_pos = len(X[(X.Sex == 0) & (y == 1)])
    len_men_neg = len(X[(X.Sex == 0) & (y == -1)])
    len_women_pos = len(X[(X.Sex == 1) & (y == 1)])
    len_women_neg = len(X[(X.Sex == 1) & (y == -1)])

    W[0][1] = (len_men*len_pos)/(D*len_men_pos)
    W[0][-1] = (len_men*len_neg)/(D*len_men_neg)

    W[1][1] = (len_women*len_pos)/(D*len_women_pos)
    W[1][-1] = (len_women*len_neg)/(D*len_women_neg)
    
    sample_weight = []
    for i in range(X.shape[0]):
        sample_weight.append(W[X.iloc[i]['Sex']][y.iloc[i]])

    return sample_weight

In [15]:
sample_weight = calc_reweight(X_train, y_train)

## Parte 3: Comparando os modelos

In [16]:
optimize_metrics = {'accuracy': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': []},
                'equal_opportunity': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': []},
                'p_percent': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': []},
                'c_variation': {'metrics': ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation'],
                'LogReg': [],
                'RegEqual': [],
                'RegDemo': [],
                'RegRewe': [],
                'RegMoo': []}}

In [17]:
metrics = ['accuracy', 'equal_opportunity', 'p_percent', 'c_variation']

In [18]:
methods = ['LogReg', 'RegEqual', 'RegDemo', 'RegRewe', 'RegMoo']
index = [sum([[method]*len(metrics) for method in methods],[]),
        sum([metrics for method in methods],[])]
results_test = pd.DataFrame(index=index, columns=metrics)
results_val = pd.DataFrame(index=index, columns=metrics)

In [19]:
# RegLog

for metric in metrics:
    reg_log = FindCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric).tune()
    # RegMoo
    print('Metric: ', metric)
    print('  Train: ')
    print('    Acc: ', reg_log.score(X_val, y_val))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_log, X_val, y_val))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_log, X_val))
    print('    CVar: ', coefficient_of_variation(reg_log, X_val, y_val))
    print('  Test: ')
    print('    Acc: ', reg_log.score(X_test, y_test))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_log, X_test, y_test))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_log, X_test))
    print('    CVar: ', coefficient_of_variation(reg_log, X_test, y_test))
    print('------------------------------------------------------')
    optimize_metrics[metric]['LogReg'] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]
    results_test.loc[('LogReg',metric),:] = [reg_log.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_log, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_log, X_test, y_test)]

Metric:  accuracy
  Train: 
    Acc:  0.67
    Equal:  0.7368421052631579
    Parid:  0.7948051948051948
    CVar:  0.681929875711223
  Test: 
    Acc:  0.615
    Equal:  0.9031954887218046
    Parid:  0.9696601941747574
    CVar:  0.6774767522093116
------------------------------------------------------
Metric:  equal_opportunity
  Train: 
    Acc:  0.61
    Equal:  1.0
    Parid:  0.9551020408163265
    CVar:  0.4962464564310949
  Test: 
    Acc:  0.56
    Equal:  1.0
    Parid:  0.9859943977591036
    CVar:  0.4874980215217845
------------------------------------------------------
Metric:  p_percent
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4814827591488289
------------------------------------------------------
Metric:  c_variation
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535
    Equal:  1.0
    Parid:  

In [20]:
# RegEqual

for metric in metrics:
    reg_equal = FindCCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric, base_model='equal').tune()
    print('Metric: ', metric)
    print('  Train: ')
    print('    Acc: ', reg_equal.score(X_val, y_val))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_equal, X_val, y_val))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_equal, X_val))
    print('    CVar: ', coefficient_of_variation(reg_equal, X_val, y_val))
    print('  Test: ')
    print('    Acc: ', reg_equal.score(X_test, y_test))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_equal, X_test, y_test))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_equal, X_test))
    print('    CVar: ', coefficient_of_variation(reg_equal, X_test, y_test))
    print('------------------------------------------------------')
    optimize_metrics[metric]['RegEqual'] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]
    results_test.loc[('RegEqual',metric),:] = [reg_equal.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_equal, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_log, X_test),
                            coefficient_of_variation(reg_equal, X_test, y_test)]

Metric:  accuracy
  Train: 
    Acc:  0.7
    Equal:  0.9800000000000001
    Parid:  0.9236111111111112
    CVar:  0.5615127479202975
  Test: 
    Acc:  0.59
    Equal:  0.9517543859649122
    Parid:  0.9252243270189431
    CVar:  0.5726365997985334
------------------------------------------------------
Metric:  equal_opportunity
  Train: 
    Acc:  0.59
    Equal:  1.0
    Parid:  0.9778672032193159
    CVar:  0.49330307127033185
  Test: 
    Acc:  0.545
    Equal:  1.0
    Parid:  0.9852941176470589
    CVar:  0.48401180478149647
------------------------------------------------------
Metric:  p_percent
  Train: 
    Acc:  0.65
    Equal:  0.9058823529411764
    Parid:  0.9970845481049563
    CVar:  0.7406041050293549
  Test: 
    Acc:  0.62
    Equal:  0.9569838056680162
    Parid:  0.91375
    CVar:  0.7065987656161303
------------------------------------------------------
Metric:  c_variation
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  T

In [21]:
# RegDemo

for metric in metrics:
    reg_demo = FindCCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric, base_model='demographic').tune()
    print('Metric: ', metric)
    print('  Train: ')
    print('    Acc: ', reg_demo.score(X_val, y_val))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_demo, X_val, y_val))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_demo, X_val))
    print('    CVar: ', coefficient_of_variation(reg_demo, X_val, y_val))
    print('  Test: ')
    print('    Acc: ', reg_demo.score(X_test, y_test))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_demo, X_test, y_test))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_demo, X_test))
    print('    CVar: ', coefficient_of_variation(reg_demo, X_test, y_test))
    print('------------------------------------------------------')
    optimize_metrics[metric]['RegDemo'] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]
    results_test.loc[('RegDemo',metric),:] = [reg_demo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_demo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_demo, X_test),
                            coefficient_of_variation(reg_demo, X_test, y_test)]

Metric:  accuracy
  Train: 
    Acc:  0.7
    Equal:  0.9800000000000001
    Parid:  0.9236111111111112
    CVar:  0.5615127479202975
  Test: 
    Acc:  0.59
    Equal:  0.9517543859649122
    Parid:  0.9252243270189431
    CVar:  0.5726365997985334
------------------------------------------------------
Metric:  equal_opportunity
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4814827591488289
------------------------------------------------------
Metric:  p_percent
  Train: 
    Acc:  0.65
    Equal:  0.9058823529411764
    Parid:  0.9970845481049563
    CVar:  0.7406041050293549
  Test: 
    Acc:  0.615
    Equal:  0.9412955465587044
    Parid:  0.922979797979798
    CVar:  0.7164875847013931
------------------------------------------------------
Metric:  c_variation
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535

In [22]:
# RegRewe

for metric in metrics:
    reg_rewe = FindCLogisticRegression(X_train, y_train, X_val, y_val, metric=metric, sample_weight=sample_weight).tune()
    print('Metric: ', metric)
    print('  Train: ')
    print('    Acc: ', reg_rewe.score(X_val, y_val))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_rewe, X_val, y_val))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_rewe, X_val))
    print('    CVar: ', coefficient_of_variation(reg_rewe, X_val, y_val))
    print('  Test: ')
    print('    Acc: ', reg_rewe.score(X_test, y_test))
    print('    Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_rewe, X_test, y_test))
    print('    Parid: ', p_percent_score(sensitive_column="Sex")(reg_rewe, X_test))
    print('    CVar: ', coefficient_of_variation(reg_rewe, X_test, y_test))
    print('------------------------------------------------------')
    optimize_metrics[metric]['RegRewe'] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]
    results_test.loc[('RegRewe',metric),:] = [reg_rewe.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_rewe, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_rewe, X_test),
                            coefficient_of_variation(reg_rewe, X_test, y_test)]

Metric:  accuracy
  Train: 
    Acc:  0.7
    Equal:  0.9800000000000001
    Parid:  0.9236111111111112
    CVar:  0.5615127479202975
  Test: 
    Acc:  0.59
    Equal:  0.9517543859649122
    Parid:  0.9252243270189431
    CVar:  0.5726365997985334
------------------------------------------------------
Metric:  equal_opportunity
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4814827591488289
------------------------------------------------------
Metric:  p_percent
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4814827591488289
------------------------------------------------------
Metric:  c_variation
  Train: 
    Acc:  0.57
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4896103794185822
  Test: 
    Acc:  0.535
    Equal:  1.0
    Parid:  1.0
    CVar:  0.4814827591488

In [23]:
# RegMoo
moo_learn = MOOLogisticRegression(X_train, y_train, X_val, y_val)
moo_learn.tune()

for metric in metrics:
    reg_moo = moo_learn.tune(metric)
    print('------------------------------------------------------')
    print('Metric: ', metric)
    print('   Acc: ', reg_moo.score(X_test, y_test))
    print('   Equal: ', equal_opportunity_score(sensitive_column="Sex")(reg_moo, X_test, y_test))
    print('   Parid: ', p_percent_score(sensitive_column="Sex")(reg_moo, X_test))
    print('   CVar: ', coefficient_of_variation(reg_moo, X_test, y_test))
    print('------------------------------------------------------')
    
    results_test.loc[('RegMoo',metric),:] = [reg_moo.score(X_test, y_test),
                            equal_opportunity_score(sensitive_column="Sex")(reg_moo, X_test, y_test),
                            p_percent_score(sensitive_column="Sex")(reg_moo, X_test),
                            coefficient_of_variation(reg_moo, X_test, y_test)]

Using license file /home/marcos/gurobi.lic
Academic license - for non-commercial use only


/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,


------------------------------------------------------
Metric:  accuracy
   Acc:  0.595
   Equal:  0.9653508771929824
   Parid:  0.9332003988035893
   CVar:  0.5622853233883512
------------------------------------------------------


/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,


------------------------------------------------------
Metric:  equal_opportunity
   Acc:  0.595
   Equal:  0.9653508771929824
   Parid:  0.9332003988035893
   CVar:  0.5622853233883512
------------------------------------------------------


/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,


------------------------------------------------------
Metric:  p_percent
   Acc:  0.57
   Equal:  0.989247311827957
   Parid:  0.9518229166666666
   CVar:  0.7752871433074869
------------------------------------------------------


/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:155: RuntimeWarning: invalid value encountered in double_scalars
  score = np.minimum(p_y1_z1 / p_y1_z0, p_y1_z0 / p_y1_z1)
/home/marcos/Documents/Experiments/envs/crime/lib/python3.7/site-packages/sklego/metrics.py:79: RuntimeWarning: No samples with y_hat == 1 for Sex == 1, returning 0
  RuntimeWarning,


------------------------------------------------------
Metric:  c_variation
   Acc:  0.585
   Equal:  0.9806451612903226
   Parid:  0.9872047244094488
   CVar:  0.5159049692858131
------------------------------------------------------


In [24]:
results_test

accuracy equal_opportunity p_percent c_variation
LogReg   accuracy             0.615          0.903195   0.96966    0.677477
         equal_opportunity     0.56                 1  0.985994    0.487498
         p_percent            0.535                 1         1    0.481483
         c_variation          0.535                 1         1    0.481483
RegEqual accuracy              0.59          0.951754         1    0.572637
         equal_opportunity    0.545                 1         1    0.484012
         p_percent             0.62          0.956984         1    0.706599
         c_variation          0.535                 1         1    0.481483
RegDemo  accuracy              0.59          0.951754  0.925224    0.572637
         equal_opportunity    0.535                 1         1    0.481483
         p_percent            0.615          0.941296   0.92298    0.716488
         c_variation          0.535                 1         1    0.481483
RegRewe  accuracy              0.59          0.951754  0.925224    0.572637
         equal_opportunity    0.535                 1         1    0.481483
         p_percent            0.535                 1         1    0.481483
         c_variation          0.535                 1         1    0.481483
RegMoo   accuracy             0.595          0.965351    0.9332    0.562285
         equal_opportunity    0.595          0.965351    0.9332    0.562285
         p_percent             0.57          0.989247  0.951823    0.775287
         c_variation          0.585          0.980645  0.987205    0.515905

In [25]:
def dominate(a, b):
    sense = np.array([1, 1, 1, -1])
    if all((sense*a)>=(sense*b)) and any((sense*a)>(sense*b)):
        return True
    else:
        return False

In [26]:
new_results = results_test.copy()//0.01/100
dominate_metr = [any([dominate(other, row) for other in new_results.values]) for row in new_results.values]
new_results['dominated'] = dominate_metr
new_results

accuracy equal_opportunity p_percent c_variation  \
LogReg   accuracy              0.61               0.9      0.96        0.67   
         equal_opportunity     0.56              0.99      0.98        0.48   
         p_percent             0.53              0.99      0.99        0.48   
         c_variation           0.53              0.99      0.99        0.48   
RegEqual accuracy              0.58              0.95      0.99        0.57   
         equal_opportunity     0.54              0.99      0.99        0.48   
         p_percent             0.61              0.95      0.99         0.7   
         c_variation           0.53              0.99      0.99        0.48   
RegDemo  accuracy              0.58              0.95      0.92        0.57   
         equal_opportunity     0.53              0.99      0.99        0.48   
         p_percent             0.61              0.94      0.92        0.71   
         c_variation           0.53              0.99      0.99        0.48   
RegRewe  accuracy              0.58              0.95      0.92        0.57   
         equal_opportunity     0.53              0.99      0.99        0.48   
         p_percent             0.53              0.99      0.99        0.48   
         c_variation           0.53              0.99      0.99        0.48   
RegMoo   accuracy              0.59              0.96      0.93        0.56   
         equal_opportunity     0.59              0.96      0.93        0.56   
         p_percent             0.56              0.98      0.95        0.77   
         c_variation           0.58              0.98      0.98        0.51   

                            dominated  
LogReg   accuracy               False  
         equal_opportunity      False  
         p_percent               True  
         c_variation             True  
RegEqual accuracy               False  
         equal_opportunity      False  
         p_percent              False  
         c_variation             True  
RegDemo  accuracy                True  
         equal_opportunity       True  
         p_percent               True  
         c_variation             True  
RegRewe  accuracy                True  
         equal_opportunity       True  
         p_percent               True  
         c_variation             True  
RegMoo   accuracy               False  
         equal_opportunity      False  
         p_percent               True  
         c_variation            False

In [27]:
print(new_results.to_latex())

\begin{tabular}{lllllll}
\toprule
       &             & accuracy & equal\_opportunity & p\_percent & c\_variation &  dominated \\
\midrule
LogReg & accuracy &     0.61 &               0.9 &      0.96 &        0.67 &      False \\
       & equal\_opportunity &     0.56 &              0.99 &      0.98 &        0.48 &      False \\
       & p\_percent &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
       & c\_variation &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
RegEqual & accuracy &     0.58 &              0.95 &      0.99 &        0.57 &      False \\
       & equal\_opportunity &     0.54 &              0.99 &      0.99 &        0.48 &      False \\
       & p\_percent &     0.61 &              0.95 &      0.99 &         0.7 &      False \\
       & c\_variation &     0.53 &              0.99 &      0.99 &        0.48 &       True \\
RegDemo & accuracy &     0.58 &              0.95 &      0.92 &        0.57 &       True \\
      

In [28]:
%matplotlib notebook

In [29]:
from mpl_toolkits import mplot3d 
import numpy as np 
import matplotlib.pyplot as plt 
  
  
# Creating dataset 
x = [sol.objs[0] for sol in moo_learn.moo_.solutionsList]
y = [sol.objs[1] for sol in moo_learn.moo_.solutionsList]
z = [sol.objs[2] for sol in moo_learn.moo_.solutionsList]
  
# Creating figure 
fig = plt.figure(figsize = (10, 7)) 
ax = plt.axes(projection ="3d") 
  
# Creating plot 
ax.scatter3D(x, y, z, color = "green"); 
plt.title("simple 3D scatter plot") 
  
# show plot 
plt.show() 

<IPython.core.display.Javascript object>

In [30]:
pd.DataFrame([sol.objs for sol in moo_learn.moo_.solutionsList])

,0,1,2
0,0.618833,0.668978,0.035977
1,0.824842,0.563153,0.771585
2,0.693147,0.693147,0.000000
3,0.674379,0.585213,0.054315
4,0.621441,0.603377,0.105529
...,...,...,...
145,0.638304,0.628538,0.000813
146,0.631368,0.631026,0.002821
147,0.641806,0.619458,0.000929
148,0.688168,0.669028,0.000005


In [31]:
pd.DataFrame([sol.objs for sol in moo_learn.moo_.solutionsList])

,0,1,2
0,0.618833,0.668978,0.035977
1,0.824842,0.563153,0.771585
2,0.693147,0.693147,0.000000
3,0.674379,0.585213,0.054315
4,0.621441,0.603377,0.105529
...,...,...,...
145,0.638304,0.628538,0.000813
146,0.631368,0.631026,0.002821
147,0.641806,0.619458,0.000929
148,0.688168,0.669028,0.000005
